# Прогнозируем временные ряды продаж товаров
В этом задании предлагается проанализировать датасет продаж различных товаров и построить предсказание на 24 дня вперед с `2018-12-07`.

В ноутбуке содержится ряд вопросов, которые подскажут, на что стоит обратить внимание при прогнозировании. Ответы на них следует оформлять текстом и иллюстрировать кодом/визуализациями.


Подробное описание датасетов содержится в файле `dataset/README.md`.

<b>При выполнении задания нельзя использовать библиотеку etna.</b>
<!-- Кроме самих целевых временных рядов, можно будет использовать исторические данные о проведенных акциях.  
[Подробное описание датасета](https://data.world/data-society/causal-effects-in-time-series) -->

***
# 0. Загрузка датасета


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import ipywidgets as widgets

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Показать код", True: "Скрыть код"}
def toggle_code(state):
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))
def button_action(value):
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
state = False
toggle_code(state)
button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
display(button)

In [ ]:
!ls dataset

In [ ]:
import pandas as pd

# загружаем датасет с продажами продуктов
df_products = pd.read_csv("dataset/products.csv", index_col=0)

# загружаем датасет с рекламными акциями
df_promotions = pd.read_csv("dataset/promotions.csv", index_col=0)

In [ ]:
df_products.tail(5)

In [ ]:
print(f"Датасет с продажами: {df_products.shape}")
print(f"Датасет с акциями: {df_promotions.shape}")

In [ ]:
df_promotions.tail(5)

In [ ]:
!pip install plotly

In [ ]:
#from IPython.display import HTML
#from IPython.display import display
# Взято https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
#tag = HTML('''<script>
#code_show=true; 
#function code_toggle() {
#    if (code_show){
#        $('div.cell.code_cell.rendered.selected div.input').hide();
#    } else {
#        $('div.cell.code_cell.rendered.selected div.input').show();
#    }
#    code_show = !code_show
#} 
#$( document ).ready(code_toggle);
#</script>
#Чтобы показать/скрыть данную ячейку кода нажмите <a href="javascript:code_toggle()">здесь</a>.''')
#display(tag)


### Посмотрим, как выглядят ряды, которые нужно будет спрогнозировать

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
for products in df_products.columns[0:10]:
    fig.add_trace(go.Scatter(x=df_products.index, y=df_products[products], name = products))
    fig.update_xaxes(
        tickformat="%b\n%Y")
fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Дата", 
                 title_standoff = 25, 
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1м", step="month", stepmode="backward"),
                         dict(count=6, label="6м", step="month", stepmode="backward"),
                         dict(count=1, label="1г", step="year", stepmode="backward"),
                         dict(label="все", step="all")])))
fig.update_yaxes(title_text = "Продажи", title_standoff = 25)
fig.update_layout(title = "Набор временных рядов")
fig.show()

fig = go.Figure()
for products in df_products.columns[20:30]:
    fig.add_trace(go.Scatter(x=df_products.index, y=df_products[products], name = products))
    fig.update_xaxes(
        tickformat="%b\n%Y")
fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Дата", 
                 title_standoff = 25, 
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1м", step="month", stepmode="backward"),
                         dict(count=6, label="6м", step="month", stepmode="backward"),
                         dict(count=1, label="1г", step="year", stepmode="backward"),
                         dict(label="все", step="all")])))
fig.update_yaxes(title_text = "Продажи", title_standoff = 25)
fig.update_layout(title = "Набор временных рядов")
fig.show()

fig = go.Figure()
for products in df_products.columns[10:19]:
    fig.add_trace(go.Scatter(x=df_products.index, y=df_products[products], name = products))
    fig.update_xaxes(
        tickformat="%b\n%Y")
fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Дата", 
                 title_standoff = 25, 
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1м", step="month", stepmode="backward"),
                         dict(count=6, label="6м", step="month", stepmode="backward"),
                         dict(count=1, label="1г", step="year", stepmode="backward"),
                         dict(label="все", step="all")])))
fig.update_yaxes(title_text = "Продажи", title_standoff = 25)
fig.update_layout(title = "Набор временных рядов")
fig.show()

### Предварительно понятно, что временные ряды обладают ярко-выраженной сезонностью, вероятно коррелируют друг с другом или автокоррелируют. Кроме того, в рядах отсутствует долговременный тренд.

***
# 1. EDA
В этом блоке предлагается изучить исходные данные и ответить на вопросы про них.

## 1.1. Сезонности и тренды
<ol>
    <li>Есть ли в данных явно выраженные тренды?</li>
    <li>Есть ли в данных сезонность? Если есть, то какая? Почему это может быть важно?</li>
</ol>

Для проверки временных рядов на стационарность - следоватлеьно отсутствие сезонности, проведём для каждого временного ряда тест Дики-Фулера и визуализируем: 


* Если p-val < 0.01 - <font color='green'>зелёным</font>


* Если p-val < 0.05 - <font color='orange'>жёлтым</font>


* Если p-val > 0.05, но ADF < Critical val. - <font color='purple'>фиолетовым</font>


* Если p-val > 0.05 - <font color='red'>красным</font>

In [ ]:
# докажем отсутствие тренда првоеркой на стационарность при помощи теста Дики-Фулера
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import seaborn as sns   
import math

i,j=0,0
PLOTS_PER_ROW = 3
fig, axs = plt.subplots(math.ceil(len(df_products.columns)/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(20, 60))

for col in df_products.columns:
    result = adfuller(df_products[col])
    significance_level = 0.05
    adf_stat = result[0]
    p_val = result[1]
    crit_val_1 = result[4]['1%']
    crit_val_5 = result[4]['5%']
    crit_val_10 = result[4]['10%']
    if (p_val < significance_level) & ((adf_stat < crit_val_1)):
        linecolor = 'forestgreen' 
    elif (p_val < significance_level) & (adf_stat < crit_val_5):
        linecolor = 'orange'
    elif (p_val < significance_level) & (adf_stat < crit_val_10):
        linecolor = 'red'
    else:
        linecolor = 'purple'
    axs[i][j].plot(df_products.index, df_products[col], color=linecolor)
    axs[i][j].set_title(f'ADF Statistic {adf_stat:0.3f}, p-value: {p_val:0.3f}\nCritical Values 1%: {crit_val_1:0.3f}, 5%: {crit_val_5:0.3f}, 10%: {crit_val_10:0.3f}', fontsize=14)
    axs[i][j].set_ylabel(ylabel=col, fontsize=14)
    j+=1
    if j%PLOTS_PER_ROW==0:
        i+=1
        j=0

plt.tight_layout()
plt.show()


**На 5% уровне значимости можно сказать, что все временные ряды стационарны согласно статистике Дики-Фулера. Нет ни одного красного графика. Это значит, что в данных нет выраженного тренда.**

***
Проведём также анализ Фурье для выявления сезонностей в данных. 
Видно, что анализ не выделил каких-то больших сезонностей при таком подходе, обращая внимания только на малочастотные гармоники.

In [ ]:
# для определения сезонности мы можем провести гармонический анализ с преобразованием Фурье
import heapq
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import seaborn as sns   
import math
import numpy as np
lenth_prod = len(df_products.index)
ts_ft=np.abs(np.fft.rfft(df_products['product_10']))
ts_freq = np.fft.rfftfreq(lenth_prod)*lenth_prod
ts = pd.DataFrame(ts_ft, columns = ['ts_ft'])
ts_freq = np.fft.rfftfreq(lenth_prod)
# Посмотрим на топ 10 самых выделяющихся гармоник для каждой переменной
fig = go.Figure()
for col in df_products.columns:
    ts_ft=np.abs(np.fft.rfft(df_products[col]))
    ts_freq = np.fft.rfftfreq(lenth_prod)*lenth_prod
    ts = pd.DataFrame(ts_ft, columns = [col])
    fig.add_trace(go.Scatter(x=ts.index, y=ts[col], name = col))
fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Длина ряда", 
                 title_standoff = 25,)
fig.update_yaxes(title_text = "Частота гармоник", title_standoff = 25)
fig.update_layout(title = "Набор гармоник временных рядов")
fig.show()

# можно заметить, что у рядов выделяются частоты на 3-6 днях, меньше на 12-21 днях и последние всплески на 40-50 дней, 
#что отражает еженедельную, полумесячную и ежемесячную динамики

Посмотрим на графики автокорреляций (ACF) для каждого временного ряда в размерности до 600 дней, т.к. очевидно, что больше, чем ежегодной сезонности в данных из 3-4 лет нет. К тому же визуально это видно.


Тем не менее стоит обратить внимание на максимизацию второй и третей волны на графиках. Если 3 волна превосходит по модулю значимости вторую волну, значит в данных есть четко-выраженная ежегодная сезонность.

In [ ]:
# кроме того, для определения сезонности помогут графики автокорреляции 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
i,j=0,0
PLOTS_PER_ROW = 3
fig, axs = plt.subplots(math.ceil(len(df_products.columns)/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(20, 60))

for col in df_products.columns:
    plot_acf(df_products[col], ax=axs[i][j], lags=600)
    axs[i][j].set_ylabel(ylabel=col, fontsize=14)
    j+=1
    if j%PLOTS_PER_ROW==0:
        i+=1
        j=0

plt.show()

# в данном случае мы видим, что в рядах есть значимые автокорреляции на разных уровнях: Например product_26
# обладает ежегодной сезонностью, а product_27 полугодовой
# на основании этих данных продукты можно распределить на кластеры, для которых можно обучить отдельные модели

Основываясь на данных автокорреляций мы можем подразделить времянные ряды каждого продукта по категориям:


* 1/2 года - максимизация на второй волне (6 продуктов)


* 3/4 года - максимизация в 3 квартале (2 продукта)


* 1 year - максимизация на 3 волне (31 продукта)

In [ ]:
# мы могли бы записать некоторые условные классы для классификации лагов
lag_classes = [
    {
        "name": "1/2 year",
        "min": 120,
        "max": 200
    },
    {
        "name": "3/4 year",
        "min": 200,
        "max": 300
    },
    {
        "name": "1 year",
        "min": 300,
        "max": 400
    },
    {
        "name": "1/4 year",
        "min": 80,
        "max": 120
    }
]
# Чтобы далее щаписать функцию, классифицирующую временные ряды в зависимости от того, на каком лаге максимизируется их автокорреляция
def classify_columns_by_autocorrelation(df, lag_classes):
    autocorrelation = {}
    class_dataframes = {class_["name"]: pd.DataFrame() for class_ in lag_classes}
    for column in df.columns:
        autocorr = acf(df[column], nlags=500) # ограничим количество лагов разумным пределом
        max_autocorr = max(autocorr[80:]) # не будем использовать первые 80 лагов, поскольку они не интформативны
        max_lag = np.argmax(autocorr[80:])+80 
        for lag_class in lag_classes:
            if max_lag >= lag_class["min"] and max_lag < lag_class["max"]:
                autocorrelation[column] = {"autocorrelation": max_autocorr, "lag": max_lag, "class": lag_class["name"]}
                class_dataframes[lag_class["name"]][column] = df[column] # и запишем каждый продукт в новые датафрейм в словаре
                break
    return class_dataframes



df = df_products
class_df = classify_columns_by_autocorrelation(df, lag_classes)
class_df

In [ ]:
# это бы очень пригодилось, если бы мы вручную строили проноз для отдельных типов рядов
# однако мы можем просто передать в рнаш датафрейм значения автокорреляций на каждом лаге для каждой переменной, а затем отдать
# эти данные модели, которая сама будет искать закономерности
def add_autocorr_columns(df, cols):
    for col in cols:
        autocorr_col = col + '_autocorr'
        df[autocorr_col] = acf(df[col], nlags=df.shape[0]-1)
        
cols_to_autocorr = df_products.columns
add_autocorr_columns(df, cols_to_autocorr)
df.shape

Эти данные можно использовать для построения более точных моделей, разделяя временные ряды на группы перед моделирование и включая наборы автокорреляций или некоторые собственные ряды относительно друго друга с временным лагом, кратным сезонности.
Это может привести к понастоящему превосходному моделированию. 
Но это трудоёмко, поэтому на данный момент достаточно будет данных одного ряда и промоакций.

## 1.2. Масштаб
<ol>
    <li> Какой масштаб у рядов? </li>
    <li> Какой разброс значений внутри каждого ряда? </li>
    <li> Может ли это как-то помешать при прогнозировании? </li>
</ol>

Для просмотра более точных данных о переменных, оценке нормальности распределения и визуализации распределений, посмотрим на интерактивный отчёт, созданный в виде HTML файла. Прикреплён в репозитории.

In [ ]:
# создадим html отчёт о данных для удобства их просмотра. В отчёте представлены распределения, основные статистики, корреляции
!jupyter nbextension enable --py widgetsnbextension
from pandas_profiling import ProfileReport
report = ProfileReport(df_products, title="Обзор данных", sort="ascending")
report.to_file("ts_report.html")
report # можно заметить, что многие переменные коррелируют друг с другом, как в положительном направлении, так и в отрицательном
# кроме того, не все переменные распределены приближенно по нормальному акону распределения, что говорит о том, что не ко всем из них подойдут стандартные параметрические оценки и нормы стандартизации

Можно говорить о том, что не все временные ряды обладают нормальным распределением. некоторые ряды смещены влево. Однако, допуская большой размер выборки, мы условно примем распределение переменных близко к нормальному. Хотя, конечно, в реальных продуктах необходимо будет скорее снова разбить продукты на группы и приводить распределение к нормальному средсвами логарифмирования, например. 

In [ ]:
import gc
import sys
import warnings
from joblib import Parallel, delayed
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import heapq
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
import seaborn as sns   
import math

In [ ]:
# построим периодограммы для каждого продукта, чтобы оценить масштаб сезонностей во временных рядах
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="forestgreen")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 12, 52])
    ax.set_xticklabels(
        [
            "Годовой",
            "Полугодовой",
            "Квартальный",
            "Месячный",
            "Недельный",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Спектральная плотность")
    return ax

i,j=0,0
PLOTS_PER_ROW = 3
fig, axs = plt.subplots(math.ceil(len(df_products.columns)/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(20, 60))

for col in df_products.columns:
    plot_periodogram(df_products[col], ax=axs[i][j])
    axs[i][j].set_title("Периодограмма "+col)
    j+=1
    if j%PLOTS_PER_ROW==0:
        i+=1
        j=0
# как можно заметить, и как ранее показал график автокорреляций, большинство рядов обладают годовой и полугодовой сезонностью 
# так же есть и месячная сезонность
# тем не менее, необходимо сделать прогноз на 24 дня, поэтому группировать ряды по сезоным мы не будем

Здесь мы, грубо говоря, использовали тот же подход к классификации, что использовали ранее с acf. 
На основе данных периодограмм можно выделить, что полугодовой цикл больше выделяется у продукта 39, 35, 34, 32, 31, 27, 26, 24. Годовой цикл - в основном у всех остальных. В данном случае мы не выделяли 3/4 года. Можно сказать, что группы полностью идентичны. Таким образом, выясненная закономерность устойчива.
Если бы мы подбирали модель временных рядов, мы бы могули учесть значения максимизирующих лагов для каждой группы или каждого временного ряда.

## 1.3. Аномалии
<ol>
    <li>Есть ли в рядах выбросы?</li>
    <li>Как выбросы могут повлиять на прогнозирование?</li>
    <li>Что с ними можно сделать?</li>
</ol>

Для визуализации выбросов мы визуализировали z-оценки переменных и выбрали для каждой допустимый коридор. Красные точки вне этого коридора инициируются выбросами.

In [ ]:
import numpy as np
names = list(df_products)
def modify_zscore (df): # для определения выбросов посчитаем модифицированную z-оценку
    # которая показывает распределения исходя из медианы, а не среднего
    # это нужно, потому что не все наши переменные распределены близко к нормальному закону
    median = np.median(df)
    deviation_from_median = np.array(df)-median
    mad = np.median(np.abs(deviation_from_median))
    mod_zscore = (0.6745*deviation_from_median)/(mad)
    return mod_zscore


In [ ]:
import matplotlib.pyplot as plt
import math
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import CalendarFourier,DeterministicProcess
from warnings import simplefilter

# визуализируем выбросы для каждого ряда
def plot_anomaly(score_data, threshold, ax, threshold_2):
    score_data = score_data.copy().values
    ranks = np.linspace(1, len(score_data), len(score_data))
    ranks = pd.DataFrame(ranks)
    ranks.index=df_products.index
    mask_outliers = (score_data>threshold)
    mask_outliers_2 = (score_data<threshold_2)
    _, ax = plt.subplots(figsize=(15,6))
    ax.plot(ranks[mask_outliers], score_data[mask_outliers], 'o', color='red', label='Аномалии ')
    ax.plot(ranks[~mask_outliers], score_data[~mask_outliers], 'o', color='forestgreen')
    ax.plot(ranks[mask_outliers_2], score_data[mask_outliers_2], 'o', color='red')
    ax.axhline(threshold, color='red', label='Таргет-линии', alpha=0.5)
    ax.axhline(threshold_2, color='red', alpha=0.5)
    ax.legend(loc='upper right')
    plt.show()

i,j=0,0
for col in df_products.columns:
    plot_anomaly(pd.DataFrame(modify_zscore(df_products[col]), index = df_products.index), 3.5, ax=axs[i][j], threshold_2=-2.5)

Далее мы преобразовали эти точки либо в ближайшие, либо в средние из 3 квартиля распределения. Исключительно в одном продукте, дабы не удалять точки, мы применили усреднеие по 45-55 персентилям.

Полученные оценки можно видеть на графиках ниже:

In [ ]:
# обработка выбросов. Заменим их данными из соответствующих квантилей
for col in df_products.columns:
    score_data = pd.DataFrame(modify_zscore(df_products[col]), index = df_products.index)
    df_products['mask'] = score_data
    df_products[col] = df_products[col].mask(df_products['mask'] > 3.5, df_products[col].quantile(0.95))
    df_products[col] = df_products[col].mask(df_products['mask'] < -2.5, df_products[col].quantile(0.05))

score_data = pd.DataFrame(modify_zscore(df_products['product_25']), index = df_products.index)
df_products['mask'] = score_data
df_products['product_25'] = df_products['product_25'].mask(df_products['mask'] > 3.5, df_products['product_25'].quantile(0.9))
df_products['product_25'] = df_products['product_25'].mask(df_products['mask'] < -2.5, df_products['product_25'].quantile(0.05))

score_data = pd.DataFrame(modify_zscore(df_products['product_36']), index = df_products.index)
df_products['mask'] = score_data
df_products['product_36'] = df_products['product_36'].mask(df_products['mask'] > 3.5, df_products['product_36'].quantile(0.90))
df_products['product_36'] = df_products['product_36'].mask(df_products['mask'] < -2.5, df_products['product_36'].quantile(0.45))
df_products = df_products.drop('mask', axis=1)


In [ ]:
# и снова визуализирум, чтобы убидеиться 
for col in df_products.columns: # что выбросы были преобразованы
    plot_anomaly(pd.DataFrame(modify_zscore(df_products[col]), index = df_products.index), 3.5, ax=axs[i][j], threshold_2=-2.5)

## 1.4. Взаимосвязь рядов
<ol>
    <li>Коррелируют ли ряды между собой?</li>
    <li>Можно ли как-то это использовать? Если да, как? Если нет, почему?</li>
</ol>

### Как мы выяснили ранее во время EDA, определенные ряды коррелируют между собой. 

Помимо простой парной корреляции, существует ещё такой тест, как тест рядов на причинность: 


* Тест Гренджера на причинность


* Тест Йохансена на коинтеграцию

### Тест Гренджера на причинность
Показывает, может ли один ряд быть причиной другого. По нижней оси соответственно показано, какие переменные являются причиной, а по вертикальной - какие последсвием в нижнем треугольнике. И наоборот в верхнем треугольнике.

In [ ]:
# помимо корреляции мы можем оценить, воздействуют ли определенные ряды в прошлом на другие ряды в будущем
# в этом нам поможет тест Гренджера на причинность и Йохансена на коллинеарность
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=25
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df
# функция записывает p-уровень значимости теста Гренджера на причинность. Иными словами ряды с малыми значениями (до 0,05)
# могут быть причинами других по принципе столбец_x причина строки_y.
import plotly.express as px

fig = px.imshow(grangers_causation_matrix(df_products.iloc[:, 1:30], variables = df_products.iloc[:, 1:30].columns), aspect="auto")
fig.show()

### Тест Йохансана на коинтеграцию 
Тест показывает, какие переменные могут быть коинтегрированы с другими на определённом уровне значимости и соответственно могу действитлеьно знаимо быть причинами других переменных. Здесь значения True напротив продукта значат ,что они действительно могут быть причинами других по Йохансену. В результате, совместив эти 2 теста, мы можем получить комбинации переменных, которые являются причинами друг друга.

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05):
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)
    print('       Тест Йохансена на коинтеграцию  \n', '--'*20, '\n', 'Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df_products) # можно считать достоверными оценки причинности по Гренджеру для тех переменных
# для которых оценки теста Йохансена = True

Здесь мы просто визуализирвоали эти ряды

In [ ]:
fig = go.Figure()
for products in df_products.columns[19:28]:
    fig.add_trace(go.Scatter(x=df_products.index, y=df_products[products], name = products))
    fig.update_xaxes(
        tickformat="%b\n%Y")
fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Дата", 
                 title_standoff = 25, 
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1м", step="month", stepmode="backward"),
                         dict(count=6, label="6м", step="month", stepmode="backward"),
                         dict(count=1, label="1г", step="year", stepmode="backward"),
                         dict(label="все", step="all")])))
fig.update_yaxes(title_text = "Продажи", title_standoff = 25)
fig.update_layout(title = "Набор причинных временных рядов")
fig.show()

И на последок посмотрим на тепловую карту многомерной автокорреляции переменных

In [ ]:

def estimate_multivariate_autocorrelation(df):
    n_cols = df.shape[1]
    autocorrelation_matrix = np.zeros((n_cols, n_cols))
    for i in range(n_cols):
        for j in range(n_cols):
            autocorrelation_matrix[i, j] = df[df.columns[i]].autocorr(lag=j)
    return autocorrelation_matrix
autocorrelation_matrix = estimate_multivariate_autocorrelation(df_products)
def create_autocorrelation_heatmap(autocorrelation_matrix, columns):
    data = go.Heatmap(z=autocorrelation_matrix,
                      x=columns,
                      y=columns,
                      hoverongaps = False,
                      colorbar=dict(title='Корреляционный коэффициент'))
    layout = go.Layout(title='Многомерная автокорреляция')
    fig = go.Figure(data=data, layout=layout)
    fig.show()

create_autocorrelation_heatmap(autocorrelation_matrix, df_products.columns)

Таким образом, многие временные ряды коррелируют с другими. Однако, очевидно, что эта корреляция обусловлена сезонностью продаж этих товаров. В ручной настройке мы могли бы разделить уже известные нам группы в словаре временных рядов из раздела " ", еще на несколько групп, по значениям корреляции между ними: от слабой до сильной. Тем не менее при моделировании благодаря нейронным сетям, мы можем оставить этот момент. И нам следует оставить этот момент, поскольку всё-равно мы не знаем, сколько продукта той или иной категории будет продано в момент n и n+1, а только в момент n-1.

## 1.5. Акции
<ol>
    <li>Что из себя представляет датасет с акциями?</li>
    <li>Как часто происходит каждая акция?</li>
    <li>Рекламная акция для какого-то продукта влияет на его продажи. Может ли она повлиять на продажи других продуктов?</li>
    <li>Есть ли акции, которые пересекаются по времени? Могут ли сразу несколько акций повлиять на один продукт?</li>
</ol>

Информация о датафрейме

In [ ]:
!pip install pycausalimpact

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import causalimpact
df_promotions.info() # информация о наборе данных промоакций

На такой "частотной" визуализации в виде ребящего телевизора можно наглядно увидеть, как акции распределены вов ремени. И что они действительно сильно переплетены, а какие-то длятся действительно долго. 

In [ ]:
# посмотрим на частоту акций во времени
df_promotions.reset_index(inplace=True) 
df_promo = df_promotions.melt(id_vars=['index'],
             var_name = 'name',
             value_name = 'promo') # преобразуем широкие данные в длинные

fig = px.scatter(df_promo, x="index", y="name", color="promo")

fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Дата", 
                 title_standoff = 25, 
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1м", step="month", stepmode="backward"),
                         dict(count=6, label="6м", step="month", stepmode="backward"),
                         dict(count=1, label="1г", step="year", stepmode="backward"),
                         dict(label="все", step="all")])))
fig.update_layout(
    autosize=False,
    width=800,
    height=800)
fig.show() 
# как можно заметить, есть множество пересечений, частота промоакций различная, они пересекаются
# отсюда можно сделать вывод, что акции могут воздействовать на несколько продуктов сразу 
# кроме того, фундаментально, не зная природу акций, оценить их воздействие на явно сезонные переменные будет затруднительно, а большого эффекта они не окажут

Взаимосвязь переменных промоакций и продуктов продемонстрируем на корреляционной диаграмме.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
def corr(df1, df2):
    n = len(df1)
    v1, v2 = df1.values, df2.values
    sums = np.multiply.outer(v2.sum(0), v1.sum(0))
    stds = np.multiply.outer(v2.std(0), v1.std(0))
    return pd.DataFrame((v2.T.dot(v1) - sums / n) / stds / n,
                        df2.columns, df1.columns)

reshapepromo = df_promotions[:-24]
reshapepromo = reshapepromo.iloc[:, 1:1001]
df_concat_corr = corr(df_products, reshapepromo)
au_corr = df_concat_corr.unstack()
filtercorr = au_corr[((au_corr >= .6) | (au_corr <= -.6)) & (au_corr !=1.000)]
au_corr = filtercorr.unstack(level=0)
fig = px.imshow(au_corr, aspect="auto")
fig.update_layout(font=dict(size=8))
print("Топ корреляций переменных")
fig.show()

Тоже самое можно сделать отдельно для всех промоакций.

In [ ]:
def get_redundant_pairs(df):
    '''Без дублирования'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, max, min):
    '''Получим топ коррелируемых переменных'''
    au_corr = df.corr().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    filtercorr = au_corr[((au_corr >= max) & (au_corr <= min)) | ((au_corr <= - max ) & (au_corr >= - min)) & (au_corr !=1.000)]
    au_corr = filtercorr.unstack(level=0)
    fig = px.imshow(au_corr, aspect="auto")
    fig.update_layout(font=dict(size=8))
    fig.show()

print("Топ очень высоких корреляций переменных")
get_top_abs_correlations(reshapepromo, 0.9, 0.999999)
print("Топ высоких корреляций переменных")
get_top_abs_correlations(reshapepromo, 0.7, 0.899999)
print("Топ средних корреляций переменных")
get_top_abs_correlations(reshapepromo, 0.5, 0.699999)

Таким образом, можно понять, что огромное количество промоакций связанно друг с другом и лишь некоторые влияют на продукты.
Посмотрим на таблицу промоакций.

In [ ]:
df_promotions.head(5)

Таблица не очень наглядная, но в ней видно, что из себя представляют данные - 0, когда акций нет, и 1 - когда они есть. Мы можем посчитать, какова длительность каждой акции в промежуток времени её присутствия. Запишем новый датафрейм и визуализируем ниже.

In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
df_promo = df_promotions
def calculate_promotion_duration(df, promotion_cols):
    duration_cols = [col + '_duration' for col in promotion_cols]
    df[duration_cols] = 0
    for col, duration_col in zip(promotion_cols, duration_cols):
        promotion = df[col].to_numpy()
        start = -1
        for i in range(len(promotion)):
            if start == -1:
                if promotion[i] == 1:
                    start = i
            else:
                if promotion[i] == 0:
                    end = i
                    duration = end - start
                    df.loc[start:end-1, duration_col] = duration
                    start = -1
        if start != -1:
            end = len(promotion)
            duration = end - start
            df.loc[start:, duration_col] = duration
    return df

promotion_cols = list(df_promo.columns)
calculate_promotion_duration(df_promo, promotion_cols)

In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
import plotly.express as px
fig = px.line(df_promo.iloc[:, 1002:2002], x=df_promo.index, y=list(df_promo.iloc[:, 1002:2002].columns), labels={'x':'Date','y':'Duration (days)'}, title="Promotion duration over time")
fig.show()

На таком графике можно даже выделить разные частоты промоакций. Например акции 58 и 978 длятся около 1000 дней. 
акций, которые длятся более 400 дней меньше, их можно посмотреть на глаз. 
И большая плотность акций начинается там, где из длительность меньше 400 дней. Здесь уже поможет интерактивность plotly. 
Тем не менее, разбираться вручную со всем этим разнообразием будет ужасно сложно, долго и непродуктивно. Однако, все новые переменные, которые мы уже собрали, можно "скормить" нейросети для лучших предсказаний.

## 1.6. Ваш ход
Может быть, есть еще что-то интересное, чего мы не заметили? :)

Уже выложил всё выше :)


***
# 2. Прогнозирование
В этом блоке предлагается построить прогноз на указанный промежуток времени и ответить на вопросы о метриках и валидации результатов.

<i>Возможно, в этом блоке не получится разбить код на предложенные части; в таком случае следует оставить максимально подробный комментарий к происходящему. </i>

In [ ]:
HORIZON = 24 # горизонт прогнозирования

## 2.1. Пайплайн прогнозирования
### 2.1.1. Подготовка данных
<ol>
    <li>Нужно ли как-то предобрабатывать ряды из датасета?</li>
    <li>Какие признаки можно выделить из данных?</li>
    <li>Какие признаки можно извлечь из индекса timestamp?</li>
    <li>Как использовать данные об акциях?</li>
    <li>*Есть ли среди выделенных признаков categorical признаки? Если есть, как с ними работать?</li>
</ol>

### 2.1.2. Модель
Какие модели прогнозирования могут помочь в нашей задаче? В чем их особенности, плюсы и минусы?

За основу возьмём датафрейм промоакций, добавленный их длительностью на предыдущем шаге. Но исключим последние 24 наблюдения для тренировки нейронной сети на известных данных.

In [ ]:
import numpy as np
import pandas as pd
import math
import h5py
import fix_yahoo_finance as yf
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as pdr
from time import sleep
from datetime import datetime as dt
import talib as tb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler


# Libraries required by FeatureSelector()
import lightgbm as lgb
import gc
from itertools import chain
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import copy
from matplotlib.dates import (DateFormatter, WeekdayLocator, DayLocator, MONDAY)

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, LSTM, GRU
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU, GlobalAveragePooling1D
from keras import regularizers
from keras import backend as K

In [ ]:
# Итак, выделим все предикторы в отдельный датафрейм
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(df_promo.iloc[:, 1002:])
duration_norm = pd.DataFrame(scaler.transform(df_promo.iloc[:, 1002:]), columns=df_promo.iloc[:, 1002:].columns).iloc[:,:1000]

predictors = pd.DataFrame(df_promo.iloc[:, :1000])
predictors[duration_norm.columns] = duration_norm

df_X = pd.DataFrame(predictors).iloc[:1071,:]
df_X.set_index("index", inplace=True)
df_X

Далее мы добавим в этот датафрейм: 


* День в году - рассчитанный из индекса. Так нейронная сеть поймёт ежегодную сезонность.


Отдельно в функции


* Добавим в обучающие данные временной ряд непосредственно продукта, для которого строится нейросеть, сдвинутую на 24 шага вперёд.
Таким образом мы сможем затем предсказать данные на 24 шага вперёд, согласно заданию.



* Удалим первые 24 пустых наблюдения.



И построим модель

In [ ]:
#from sklearn.model_selection import train_test_split
#train_X, test_X, train_y, test_y = train_test_split(df_X, df_Y, test_size=0.2, random_state=42)

In [ ]:
df_Y = df_products
train_X = df_X
train_X['date'] = train_X.index
train_X['dayofyear'] = pd.to_datetime(train_X['date']).dt.dayofyear
train_X = train_X.drop(columns=['date'])
train_y = df_Y

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
def build_rnn_model(train_X):
    # design network
    print("\n")
    print("RNN LSTM model architecture >")
    model = Sequential()
    model.add(LSTM(128, kernel_initializer='random_uniform',
                   bias_initializer='zeros', return_sequences=True,
                   input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dropout(0.25))
    model.add(LSTM(64, kernel_initializer='random_uniform',
                   # kernel_regularizer=regularizers.l2(0.001),
                   # activity_regularizer=regularizers.l1(0.001),
                   bias_initializer='zeros'))
    model.add(Dropout(0.25))
    model.add(Dense(1))
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0002)
    # optimizer = keras.optimizers.Adagrad(learning_rate=0.03, epsilon=1e-08, decay=0.00002)
    # optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    # optimizer = keras.optimizers.Nadam(learning_rate=0.0002, beta_1=0.9, beta_2=0.999, schedule_decay=0.004)
    # optimizer = keras.optimizers.Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
    # optimizer = keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95, epsilon=None, decay=0.0)

    model.compile(loss='mae', optimizer=optimizer, metrics=['mse', 'mae'])
    model.summary()
    return model


In [ ]:
def train_models_for_each_column(train_X, train_y, model, model_type):
    # Get the column names of the dataframe
    col_names = train_y.columns
    
    # Create a dictionary to store the models for each column
    models = {}
    
    for col in col_names:
        # Extract the train_X and train_y for the current column
        train_y_col = train_y[col]
        scaler = MinMaxScaler()
        
        train_X_col = train_X
        train_X_col[col] = train_y[col].shift(24, axis = 0)
        
        train_X_col = train_X.iloc[24:]
        
        train_X_col = scaler.fit_transform(train_X_col.values)
        train_X_col = train_X_col.reshape((train_X_col.shape[0], 1, train_X_col.shape[1]))
        train_y_col = train_y_col.iloc[24:]
        train_y_col = scaler.fit_transform(train_y_col.values.reshape(-1,1))
        model = build_rnn_model(train_X_col)
        if model_type == "LSTM":
            batch_size=10
            mc = ModelCheckpoint('best_lstm_model_{}.h5'.format(col), monitor='val_loss', save_weights_only=False,
                             mode='min', verbose=1, save_best_only=True)
        elif model_type == "CNN":
            batch_size=8
            mc = ModelCheckpoint('best_cnn_model_{}.h5'.format(col), monitor='val_loss', save_weights_only=False,
                             mode='min', verbose=1, save_best_only=True)
        # fit network
        history = model.fit(
            train_X_col, train_y_col, epochs=50, batch_size=batch_size, validation_split=0.2, 
            shuffle=False, callbacks=[mc])
        models[col] = model
    
    return models


In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
models = train_models_for_each_column(train_X, train_y, build_rnn_model, "LSTM")

Теперь просто выполним те же шаги для целевых данных, чтобы предсказать движение продаж на 24 шага вперёд.

In [ ]:
scaler = MinMaxScaler()

scaler.fit(df_promo.iloc[:, 1002:])
duration_norm = pd.DataFrame(scaler.transform(df_promo.iloc[:, 1002:]), columns=df_promo.iloc[:, 1002:].columns).iloc[:,:1000]

predictors = pd.DataFrame(df_promo.iloc[:, :1000])
predictors[duration_norm.columns] = duration_norm

df_X = pd.DataFrame(predictors)
df_X.set_index("index", inplace=True)
df_X

**Сравнение оригинального ряда и наших предсказаний приведено ниже на графиках.**

In [ ]:
df_Y = df_products
train_X = df_X
train_X['date'] = train_X.index
train_X['dayofyear'] = pd.to_datetime(train_X['date']).dt.dayofyear
train_X = train_X.drop(columns=['date'])
train_y = df_Y

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# Create a subplot for each column


# Iterate through the columns of the original dataframe
for i, col in enumerate(train_y.columns, 1):
    df = pd.DataFrame()
    df['Date'] = df_X.index
    # Get the original values and predictions for the current column
    original = train_y[col]
    original = scaler.fit_transform(original.values.reshape(-1,1))
    train_y_col = train_y[col]
        
    train_X_col = train_X
    train_X_col[col] = train_y[col]
    train_X_col[col] = train_X_col[col].shift(24, axis = 0)
        
    train_X_col = train_X.iloc[24:]
        
    train_X_col = scaler.fit_transform(train_X_col.values)
    train_X_col = train_X_col.reshape((train_X_col.shape[0], 1, train_X_col.shape[1]))
    train_y_col = train_y_col.iloc[24:]
    train_y_col = scaler.fit_transform(train_y_col.values.reshape(-1,1))
    
    predictions = models[col].predict(train_X_col)
    
    df['original'] = pd.DataFrame(original)
    df['predictions'] = pd.DataFrame(predictions)
    df['predictions'] = df['predictions'].shift(24, axis = 0)
    
    fig = go.Figure()
    # Create a trace for the original values
    original_trace = go.Scatter(x=df['Date'], y=df['original'], name='Original', mode='lines', line=dict(color='purple'))
    
    # Create a trace for the predictions
    predictions_trace = go.Scatter(x=df['Date'], y=df['predictions'], name='Predictions', mode='lines', line=dict(color='green'))
    
    # Add the traces to the subplot
    fig.add_trace(predictions_trace)
    fig.add_trace(original_trace)
    fig.update_xaxes(rangeslider_visible=True, 
                 title_text = "Дата", 
                 title_standoff = 25, 
                 rangeselector=dict(
                     buttons=list([
                         dict(count=1, label="1м", step="month", stepmode="backward"),
                         dict(count=6, label="6м", step="month", stepmode="backward"),
                         dict(count=1, label="1г", step="year", stepmode="backward"),
                         dict(label="все", step="all")])))
    
    fig.update_layout(title=col + ' Orig vs Pred')
    fig.show()


In [ ]:
prediction_deviation = recovered_data_lstm.loc[test_set.index][['Predict-Y']] - original_stock_context_fs_full.loc[test_set.index][['Predict-Y']]

## 2.2. Валидация 
### 2.2.1. Метрики
<ol>
    <li>Какие метрики качества могут быть использованы в нашей задаче?</li>
    <li>В качестве метрики качества мы хотим использовать MSE; с какими проблемами мы можем столкнуться?</li>
</ol>

### 2.2.2. Кросс-валидация
Как провести кросс-валидацию?

### 2.2.3. Сравнение моделей
Предположим, мы построили несколько пайплайнов прогнозирования. Как выбрать лучший из них?
<ol>
    <li>В датасете 30 рядов, мы посчитали метрику для каждого из них, но нам надо понять, какой из пайплайнов работает лучше; как это сделать?</li>
    <li>Мы выбрали лучший из пайплайнов; можно ли еще улучшить его? Когда стоит остановиться?</li>
    <li>*Если в предыдущих частях были рассмотрены несколько пайплайнов, какой оказался лучшим? Как выглядит лучший прогноз? (Если выше был рассмотрен один пайплайн, пропустите этот пункт)</li>
</ol>

**К сожалению, ответы на все эти вопросы зашиты в нейросеть.
За исключением кросс-валидации.**

## 2.3. Использование доп данных
<ol>
    <li>Получилось ли использовать данные об акциях при построении прогнозов?</li>
    <li>Если да, помогают ли они предсказывать точнее?</li>
    <li>Как понять, какие из акций важны, а какие нет? Могут ли скоррелированные признаки помешать оценке важности признака? Что с этим делать?</li>
</ol>

**Ответы:**

1. Да, данные об акциях были использованы и даже дополнены. Однако, на моё взгляд, столько данных даже избыточно. Возможно, стоило бы взять для оценки только те промоакции, которые коррелируют с временными рядами. Это момент для улучшения. 
2. Они помогают определять всплески активности.
3. Коллинераность прмооакций можно устранить, удаляя коллинеарные переменные из данных. Для этого можно воспользвоаться VIF-тестом.

## 2.4. Production 🚀
Мы построили восхитительный пайплайн прогнозирования! Как вывести его в продакшн?

Необходимые шаги для простого продакшна будут выглядет следующим образом:

Упаковать пайплайн: setuptools, pip или conda, чтобы упаковать пайплайн в виде библиотеки. Это упростит установку и распространение пайплайна на другие машины.

Docker или другой инструмент контейнеризации, чтобы упаковать пайплайн и его зависимости в контейнер. Это упростит развертывание пайплайна на любом компьютере, поддерживающем среду выполнения контейнера.

Выберем метод развертывания, который лучше всего подходит для использования, например, с использованием облачной платформы, такой как AWS, GCP или Azure, или развертывание на локальном компьютере.

СМожно создать API: Flask или Django, чтобы создать API, который позволит пользователям взаимодействовать с пайплайном.

Настройка мониторинга: такие инструменты, как prometheus или стек ELK, для мониторинга и регистрации пайплайна, чтобы обеспечить его бесперебойную работу.

Автоматизация тестирования: unittest или pytest, чтобы написать автоматические тесты для пайплайна, чтобы убедиться, что он работает должным образом, прежде чем развертывать его в рабочей среде.

Оптимизация производительности.

Контроль версий: git, чтобы отслеживать изменения в пайплайне и при необходимости откатываться до предыдущей версии.


***
# Свободная часть
Часть для самых смелых энтузиастов в мире временных рядов! 

Здесь предлагается попробовать сделать с датасетом что-то интересное на ваш вкус. Можно попробовать сделать что-то из предложенного:
- Покрутить датасет `dataset/influence.csv`: в нем дана матрица влияния каждой акции на каждый временной ряд продаж. Как ее можно использовать? Как это может помочь при прогнозировании? 
- Поглубже погрузиться в изучение "близости" временнях рядов и попробовать использовать эти знания для прогнозирования
- Подумать о том, как можно оценить влияние какого-то внешнего фактора-признака? Допустим, у нас есть такой же датасет с продажами и мы знаем, что определенная акция должна была повлиять на конкрентный ряд. Как оценить это влияние?
- Любая другая тема, которая кажется важной и интересной

В другой раз)